In [ ]:
import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)
from scipy.constants import c

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from __future__ import division

In [ ]:
from PyHEADTAIL_feedback.core import Parameters, process, z_bins_to_bin_edges, bin_mids

In [ ]:
def generate_signal(min_val,max_val,n_points):
    bin_width = (max_val-min_val)/float(n_points-1)
    z_bins = np.linspace(min_val-bin_width/2.,max_val+bin_width/2.,n_points+1)
    
    signal_class = 0
    bin_edges = z_bins_to_bin_edges(z_bins)
    n_segments = 1
    n_bins_per_segment = n_points
    segment_ref_points = [(min_val + max_val) / 2.]
    previous_parameters = []
    location = 0. 
    beta = 1.
    
    parameters = Parameters(signal_class, bin_edges, n_segments,
               n_bins_per_segment, segment_ref_points,
               previous_parameters, location, beta)
    signal_x = bin_mids(bin_edges)
    signal_xp = np.zeros(n_points)
    
    return parameters, signal_x, signal_xp


def rotate(angle, parameters, x, xp):
    s = np.sin(angle)
    c = np.cos(angle)
    beta_x = parameters['beta']

    new_x = c * x + beta_x * s * xp
    new_xp = (-1. / beta_x) * s * x + c * xp
    
    return parameters, new_x, new_xp

def process_signals(processors, parameters, x, xp):
    parameters_out, x_out = process(parameters, x, processors)
    if np.abs(np.min(x_out)) > np.max(x_out):
        return np.min(x_out)
    else:
        return np.max(x_out)
    
    

In [ ]:
Q_x = 25. +0.285678326893 # + 0.1*np.pi
n_points = 10
min_val = 0.
max_val = 1.
n_turns = 1000

In [ ]:
# error distribution
from PyHEADTAIL_feedback.processors.register import TurnDelay

# This is a test, which tests how well the combiner roates an ideal signal. Horizontal values represent values from
# the combiner while the vectors from the origin represent values given to the register.

parameters, signal_x, signal_xp = generate_signal(min_val, max_val, n_points)

processors = [
#     TurnDelay(delay=1, tune=Q_x+0.005, n_taps=9, combiner='cosine_sum',
#                  additional_phase_advance=0)
#     TurnDelay(delay=1, tune=Q_x+0.005, n_taps=2, combiner='vector_sum',
#                  additional_phase_advance=0)
    TurnDelay(delay=1, tune=Q_x, n_taps=9, combiner='hilbert',
                 additional_phase_advance=0),
#     TurnDelay(delay=1, tune=Q_x, n_taps=9, combiner='DCrem_vector_sum',
#                  additional_phase_advance=0),
]

fig = plt.figure(figsize=(10, 12))
seq_col_brew = sns.color_palette("Blues", n_turns)
sns.set_palette(seq_col_brew)

ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

bins = np.linspace(-0.5,0.5,50)
errors = np.zeros(n_turns)

for i in xrange(n_turns):
    angle = Q_x*2.*np.pi
    parameters, signal_x, signal_xp = rotate(angle,parameters, signal_x, signal_xp)
    ax1.plot(signal_x,signal_xp, '-')
    max_x = process_signals(processors, parameters, signal_x, signal_xp)
    ax1.axvline(x=max_x,linewidth=3.0,color = seq_col_brew[i],linestyle='--')
    
    errors[i] = max_x-signal_x[-1]
    
print 'Average error: ' + str(np.mean(errors))
    
ax1.set_xlabel('Beam X ')
ax1.set_ylabel('Beam Xp ')

ax2.hist(errors, bins=bins, normed=1, facecolor='green', alpha=0.75)
ax2.set_xlabel('X value error')
ax2.set_ylabel('N turns')
plt.show()



In [ ]:
from PyHEADTAIL_feedback.signal_tools.signal_generators import SimpleBeam

from PyHEADTAIL_feedback.signal_tools.plotters import plot_traces, plot_3D_traces, plot_beam

In [ ]:
from PyHEADTAIL_feedback.processors.register import TurnDelay
from PyHEADTAIL_feedback.processors.misc import Bypass

In [ ]:
beam = SimpleBeam(n_bunches=3, bunch_spacing=25e-9, bunch_length=4e-9, intensity=1e11, n_slices=5)

def offset_kicker(seed):
    kick = np.ones(len(seed))*1e-3
    
    return kick

processors = [
    # Bypass()
    TurnDelay(delay=1, tune=Q_x, n_taps=4, combiner='vector_sum',
                 additional_phase_advance=0)
]

angle = Q_x*2.*np.pi
    

fig = plt.figure(figsize=(10, 16))
seq_col_brew = sns.color_palette("Blues", n_turns)
sns.set_palette(seq_col_brew)

ax1 = fig.add_subplot(211)

beam.x = beam.z/np.max(beam.z)* 1.

for i in xrange(n_turns):
    angle = Q_x*2.*np.pi
    beam.rotate(angle)
    parameters_in, signal_in = beam.signal()
    
    parameters_out, signal_out = process(parameters_in,np.copy(signal_in),processors)
    
    if np.abs(np.min(signal_out)) > np.max(signal_out):
        max_signal = np.min(signal_out)
    else:
        max_signal = np.max(signal_out)
    x = beam.x
    xp = beam.xp
    ax1.plot(x,xp, '.')
    ax1.axvline(x=max_signal,linewidth=3.0,color = seq_col_brew[i],linestyle='--')

ax1.set_xlabel('Beam X ')
ax1.set_ylabel('Beam Xp ')
plt.show()
